## Hyperparameters: Influence and Optimization

This notebook contains code for

1. Influence of various Hyperparameters on Trainings process: input/output sequence length, learning rate, regularization, optimizer

2. Hyperparameteroptimization using RandomSearch

2. 1 Optimization P-Model

2. 2 Optimization PF-Model

In order to keep this notebook clearly readable, some functions are outsourced in utils/

____

### Imports

In [ ]:
import pandas as pd
import time
import pickle

from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout
from keras.regularizers import l1, l2, l1_l2
from kerastuner import HyperModel, RandomSearch, HyperParameters
from keras.optimizers import Adam, Adagrad, Adadelta, SGD

from livelossplot import PlotLossesKeras

import warnings
warnings.filterwarnings('ignore')

____

#### Load data

In [ ]:
from utils.data_utils import data_loader
from utils.utils import train_test_val_data
import config

# load data
data = data_loader(config.columns_P)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)
df_scaled = pd.DataFrame(scaled_data, columns=data.columns)


X_train, X_val, X_test, y_train, y_val, y_test = train_test_val_data(df_scaled, 
                                                                     len(data.index.unique()), 
                                                                     1)

____

#### 1. Influence of various Hyperparameters on Trainings process

In [ ]:
from utils.plot_utils import plot_validation_losses_and_durations

#### 1.1 Influence input sequence length

In [ ]:
units_0 = 64
units_outputs = y_train.shape[1]
batch_size = 32

curves_sequences = []
train_durations = []

for sequence_length in [1,3,6]:
    len_dataset = len(data.index.unique())
    num_target_variables = 1

    start_time = time.time()

    X_train, X_val, X_test, y_train, y_val, y_test = train_test_val_data(df_scaled, len_dataset, num_target_variables, sequence_length=sequence_length)

    model = Sequential()

    # Eingabeschicht
    model.add(GRU(units=units_0,            # Hyperparameter -> kann variiert und angepasst werden
                return_sequences=False,   # Konfigurationsparameter, default: False
                                            # Funktionalität:   bestimmt, ob die Schicht einen Ausgabevektor für jeden Zeitpunkt in der Eingabesequenz (return_sequences=True) 
                                            #                   oder nur für den letzten Zeitpunkt (return_sequences=False) zurückgeben soll
                                            #   False: gibt Ausgabevektor für den letzten Zeitschritt zurück: (Anzahl der Beispiele, Anzahl der Units)
                                            #   True:  gibt Ausgabevektor für jeden Zeitschritt in der Eingabesequenz zurück: (Anzahl der Beispiele, Anzahl der Zeitschritte, Anzahl der Units)
                                            # Anwendung: return_sequences=True: mehrere rekurrente Schichten hintereinander (damit jede Schicht eine Sequenz an die nächste weitergibt),  Ausgabe des Modells ist selbst eine Sequenz; 
                                            #            ansonsten: return_sequences=False.
                                            # 
                input_shape=(
                    X_train.shape[1],           # Sequenzlänge
                    X_train.shape[2]            # Anzahl der Features
                    )
                )
            )

    # Ausgabeschicht
    model.add(Dense(units_outputs, activation='linear'))

    # Konfigurieren des Modells für das Training -> Festlegung der Lernart sowie die Bewertung des Trainingsprozesses
    model.compile(
        optimizer='adam',                   # der Optimizer ist ein Algorithmus zur Aktualisierung des Netzwerks, wobei die Gewichte des Modells so angepasst werden, dass Verluste minimiert werden
                                            # Verschiedene Optimierer haben unterschiedliche Eigenschaften:
                                            # Adam - adaptive moment estimation: Grundprinzipien
                                            #   - Adaptive Lernraten: Lernrate wird für jeden Parameter individuell angepasst, basierend auf der Schätzung des ersten Mittelwert und des zweiten Moments der Gradienten
                                            #   - Moment-Schätzungen:   > erstes Moment (Mittelwert): Berechnung expontentiell abnehmender Durchschnittswerte vergangener Gradienten -> Steuerung zu relevanten Richtung des Gradientenabstiegs
                                            #                           > zweites Moment(Varianz): Berechnung exponentiuell abnehmender Durchschnittswerte vergangener quadrierter Gradienten 
                                            #                                                       -> Adaption der Lernrate, Regulierung der Schrittgröße basierend auf der Unsicherheit des Gradienten
                                            #   - Korrektur der Bias: Verhinderung der Tendenz, das Schätzungen zu Beginn gegen 0 gehen
                                            # Vorteile: Effizienz, wenigeer manuelle Einstellung der Lernrate, gute Performance bei großen Datenmengen/vielen Parametern
        loss='mean_squared_error',          # Verlustfunktion, misst die Genauigkeit des Modells. MSE misst die durchschnittliche quadratische Abweichung zwischen den vorhergesagten und den tatsächlichen Werten
        metrics=['mean_absolute_error']      # Metriken, die für das Training bewertet werden sollen, weitere Alternativen: 'accuracy', ...
        )
    
    # Trainieren des Modells
    history = model.fit(
        X_train, y_train,                   # Übergabe der Trainingsdaten
        epochs=30,                          # Anzahl der Durchläufe des gesamten Trainingsdatensatzes
                                            #   -> Einfluss: Mehr Epochen können zu einer besseren Anpassung des Modells führen <-> Gefahr des Overfittings
        batch_size=batch_size,                      # Bestimmt die Anzahl der verwendeten Datenpunkte für eine Iteration, bevor die Modellgewichte aktualisiert werden
                                            #   -> größere Batch-Größen: stabilere, aber langsamer konvergierende Updates <-> kleinere Batch-Größen: schnellere, weniger stabile Updates
        validation_data=(X_val, y_val),     # Validierungsdaten, ermöglichen die Überwachung des Trainingsprozesses -> Erkennung von Overfitting
        callbacks=[PlotLossesKeras()],
        use_multiprocessing=True,           # Laufzeitoptimierung
        workers=4,                          # Nutzen mehrerer CPU-Kerne
        verbose=1,                          # Steuert die Menge an Infos, welche während des Trainings ausgegeben werden -> verbose=1 zeigt den Fortschritt für jede Epoche an
    )

    curves_sequences.append(history)

    end_time = time.time()
    training_duration = end_time - start_time  # Dauer in Sekunden

    train_durations.append(training_duration)

with open('data/hyper_analysis/21_curves.pkl', 'wb') as f:
    pickle.dump(curves_sequences, f)

with open('data/hyper_analysis/21_durations.pkl', 'wb') as f:
    pickle.dump(train_durations, f)

In [ ]:
with open('data/hyper_analysis/21_curves.pkl', 'rb') as f:
    curves_sequences = pickle.load(f)

with open('data/hyper_analysis/21_durations.pkl', 'rb') as f:
    train_durations = pickle.load(f)
    
plot_validation_losses_and_durations(curves_sequences, ["1 Tag", "3 Tage", "6 Tage"], train_durations)

#### 2.2 Influence prediction length

In [ ]:
units_0 = 64
units_outputs = y_train.shape[1]
batch_size = 32

curves_prediction_length = []
train_durations_2 = []

for prediction_length in [(1/24),(6/24),(12/24),1]:
    len_dataset = len(data.index.unique())
    num_target_variables = 1

    start_time = time.time()

    X_train, X_val, X_test, y_train, y_val, y_test = utils.train_test_val_data(df_scaled, len_dataset, num_target_variables, prediction_length=prediction_length)

    units_outputs = y_train.shape[1]
    
    model = Sequential()

    # Eingabeschicht
    model.add(GRU(units=units_0,            # Hyperparameter -> kann variiert und angepasst werden
                return_sequences=False,   # Konfigurationsparameter, default: False
                                            # Funktionalität:   bestimmt, ob die Schicht einen Ausgabevektor für jeden Zeitpunkt in der Eingabesequenz (return_sequences=True) 
                                            #                   oder nur für den letzten Zeitpunkt (return_sequences=False) zurückgeben soll
                                            #   False: gibt Ausgabevektor für den letzten Zeitschritt zurück: (Anzahl der Beispiele, Anzahl der Units)
                                            #   True:  gibt Ausgabevektor für jeden Zeitschritt in der Eingabesequenz zurück: (Anzahl der Beispiele, Anzahl der Zeitschritte, Anzahl der Units)
                                            # Anwendung: return_sequences=True: mehrere rekurrente Schichten hintereinander (damit jede Schicht eine Sequenz an die nächste weitergibt),  Ausgabe des Modells ist selbst eine Sequenz; 
                                            #            ansonsten: return_sequences=False.
                                            # 
                input_shape=(
                    X_train.shape[1],           # Sequenzlänge
                    X_train.shape[2]            # Anzahl der Features
                    )
                )
            )

    # Ausgabeschicht
    model.add(Dense(units_outputs, activation='linear'))

    # Konfigurieren des Modells für das Training -> Festlegung der Lernart sowie die Bewertung des Trainingsprozesses
    model.compile(
        optimizer='adam',                   # der Optimizer ist ein Algorithmus zur Aktualisierung des Netzwerks, wobei die Gewichte des Modells so angepasst werden, dass Verluste minimiert werden
                                            # Verschiedene Optimierer haben unterschiedliche Eigenschaften:
                                            # Adam - adaptive moment estimation: Grundprinzipien
                                            #   - Adaptive Lernraten: Lernrate wird für jeden Parameter individuell angepasst, basierend auf der Schätzung des ersten Mittelwert und des zweiten Moments der Gradienten
                                            #   - Moment-Schätzungen:   > erstes Moment (Mittelwert): Berechnung expontentiell abnehmender Durchschnittswerte vergangener Gradienten -> Steuerung zu relevanten Richtung des Gradientenabstiegs
                                            #                           > zweites Moment(Varianz): Berechnung exponentiuell abnehmender Durchschnittswerte vergangener quadrierter Gradienten 
                                            #                                                       -> Adaption der Lernrate, Regulierung der Schrittgröße basierend auf der Unsicherheit des Gradienten
                                            #   - Korrektur der Bias: Verhinderung der Tendenz, das Schätzungen zu Beginn gegen 0 gehen
                                            # Vorteile: Effizienz, wenigeer manuelle Einstellung der Lernrate, gute Performance bei großen Datenmengen/vielen Parametern
        loss='mean_squared_error',          # Verlustfunktion, misst die Genauigkeit des Modells. MSE misst die durchschnittliche quadratische Abweichung zwischen den vorhergesagten und den tatsächlichen Werten
        metrics=['mean_absolute_error']      # Metriken, die für das Training bewertet werden sollen, weitere Alternativen: 'accuracy', ...
        )
    
    # Trainieren des Modells
    history = model.fit(
        X_train, y_train,                   # Übergabe der Trainingsdaten
        epochs=50,                          # Anzahl der Durchläufe des gesamten Trainingsdatensatzes
                                            #   -> Einfluss: Mehr Epochen können zu einer besseren Anpassung des Modells führen <-> Gefahr des Overfittings
        batch_size=batch_size,                      # Bestimmt die Anzahl der verwendeten Datenpunkte für eine Iteration, bevor die Modellgewichte aktualisiert werden
                                            #   -> größere Batch-Größen: stabilere, aber langsamer konvergierende Updates <-> kleinere Batch-Größen: schnellere, weniger stabile Updates
        validation_data=(X_val, y_val),     # Validierungsdaten, ermöglichen die Überwachung des Trainingsprozesses -> Erkennung von Overfitting
        callbacks=[PlotLossesKeras()],
        use_multiprocessing=True,           # Laufzeitoptimierung
        workers=4,                          # Nutzen mehrerer CPU-Kerne
        verbose=1,                          # Steuert die Menge an Infos, welche während des Trainings ausgegeben werden -> verbose=1 zeigt den Fortschritt für jede Epoche an
    )

    curves_prediction_length.append(history)

    end_time = time.time()
    training_duration = end_time - start_time  # Dauer in Sekunden

    train_durations_2.append(training_duration)
    
with open('data/hyper_analysis/22_curves.pkl', 'wb') as f:
    pickle.dump(curves_prediction_length, f)

with open('data/hyper_analysis/22_durations.pkl', 'wb') as f:
    pickle.dump(train_durations_2, f)

In [ ]:
with open('data/hyper_analysis/22_curves.pkl', 'rb') as f:
    curves_prediction_length = pickle.load(f)

with open('data/hyper_analysis/22_durations.pkl', 'rb') as f:
    train_durations_2 = pickle.load(f)

plot_validation_losses_and_durations(curves_prediction_length, ["1 Stunde","6 Stunden", "12 Stunden", "24 Stunden"], train_durations_2)

#### 1.3 Influence learning rate

In [ ]:
learning_rate_curves = []
train_durations_3 = []

for learning_rate in [0.01, 0.001, 0.0001, 0.00001]:
    len_dataset = len(data.index.unique())
    num_target_variables = 1
    
    start_time = time.time()

    X_train, X_val, X_test, y_train, y_val, y_test = utils.train_test_val_data(df_scaled, len_dataset, num_target_variables)
    units_outputs = y_train.shape[1]
    
    model = Sequential()

    # Eingabeschicht
    model.add(GRU(units=units_0,            # Hyperparameter -> kann variiert und angepasst werden
                return_sequences=False,   # Konfigurationsparameter, default: False
                                            # Funktionalität:   bestimmt, ob die Schicht einen Ausgabevektor für jeden Zeitpunkt in der Eingabesequenz (return_sequences=True) 
                                            #                   oder nur für den letzten Zeitpunkt (return_sequences=False) zurückgeben soll
                                            #   False: gibt Ausgabevektor für den letzten Zeitschritt zurück: (Anzahl der Beispiele, Anzahl der Units)
                                            #   True:  gibt Ausgabevektor für jeden Zeitschritt in der Eingabesequenz zurück: (Anzahl der Beispiele, Anzahl der Zeitschritte, Anzahl der Units)
                                            # Anwendung: return_sequences=True: mehrere rekurrente Schichten hintereinander (damit jede Schicht eine Sequenz an die nächste weitergibt),  Ausgabe des Modells ist selbst eine Sequenz; 
                                            #            ansonsten: return_sequences=False.
                                            # 
                input_shape=(
                    X_train.shape[1],           # Sequenzlänge
                    X_train.shape[2]            # Anzahl der Features
                    )
                )
            )

    # Ausgabeschicht
    model.add(Dense(units_outputs, activation='linear'))

    optimizer = Adam(learning_rate=learning_rate)
    
    # Konfigurieren des Modells für das Training -> Festlegung der Lernart sowie die Bewertung des Trainingsprozesses
    model.compile(
        optimizer=optimizer,                # der Optimizer ist ein Algorithmus zur Aktualisierung des Netzwerks, wobei die Gewichte des Modells so angepasst werden, dass Verluste minimiert werden
                                            # Verschiedene Optimierer haben unterschiedliche Eigenschaften:
                                            # Adam - adaptive moment estimation: Grundprinzipien
                                            #   - Adaptive Lernraten: Lernrate wird für jeden Parameter individuell angepasst, basierend auf der Schätzung des ersten Mittelwert und des zweiten Moments der Gradienten
                                            #   - Moment-Schätzungen:   > erstes Moment (Mittelwert): Berechnung expontentiell abnehmender Durchschnittswerte vergangener Gradienten -> Steuerung zu relevanten Richtung des Gradientenabstiegs
                                            #                           > zweites Moment(Varianz): Berechnung exponentiuell abnehmender Durchschnittswerte vergangener quadrierter Gradienten 
                                            #                                                       -> Adaption der Lernrate, Regulierung der Schrittgröße basierend auf der Unsicherheit des Gradienten
                                            #   - Korrektur der Bias: Verhinderung der Tendenz, das Schätzungen zu Beginn gegen 0 gehen
                                            # Vorteile: Effizienz, wenigeer manuelle Einstellung der Lernrate, gute Performance bei großen Datenmengen/vielen Parametern
        loss='mean_squared_error',          # Verlustfunktion, misst die Genauigkeit des Modells. MSE misst die durchschnittliche quadratische Abweichung zwischen den vorhergesagten und den tatsächlichen Werten
        metrics=['mean_absolute_error']      # Metriken, die für das Training bewertet werden sollen, weitere Alternativen: 'accuracy', ...
        )
    
    # Trainieren des Modells
    history = model.fit(
        X_train, y_train,                   # Übergabe der Trainingsdaten
        epochs=50,                          # Anzahl der Durchläufe des gesamten Trainingsdatensatzes
                                            #   -> Einfluss: Mehr Epochen können zu einer besseren Anpassung des Modells führen <-> Gefahr des Overfittings
        batch_size=batch_size,                      # Bestimmt die Anzahl der verwendeten Datenpunkte für eine Iteration, bevor die Modellgewichte aktualisiert werden
                                            #   -> größere Batch-Größen: stabilere, aber langsamer konvergierende Updates <-> kleinere Batch-Größen: schnellere, weniger stabile Updates
        validation_data=(X_val, y_val),     # Validierungsdaten, ermöglichen die Überwachung des Trainingsprozesses -> Erkennung von Overfitting
        callbacks=[PlotLossesKeras()],
        use_multiprocessing=True,           # Laufzeitoptimierung
        workers=4,                          # Nutzen mehrerer CPU-Kerne
        verbose=1,                          # Steuert die Menge an Infos, welche während des Trainings ausgegeben werden -> verbose=1 zeigt den Fortschritt für jede Epoche an
    )

    learning_rate_curves.append(history)

    end_time = time.time()
    training_duration = end_time - start_time  # Dauer in Sekunden

    train_durations_3.append(training_duration)
    
with open('data/hyper_analysis/23_curves.pkl', 'wb') as f:
    pickle.dump(learning_rate_curves, f)

with open('data/hyper_analysis/23_durations.pkl', 'wb') as f:
    pickle.dump(train_durations_3, f)

In [ ]:
with open('data/hyper_analysis/23_curves.pkl', 'rb') as f:
    learning_rate_curves = pickle.load(f)

with open('data/hyper_analysis/23_durations.pkl', 'rb') as f:
    train_durations_3 = pickle.load(f)

plot_validation_losses_and_durations(learning_rate_curves, ["0.1","0.01", "0.001", "0.0001"], train_durations_3)

In [ ]:
learning_rate_2_curves = []
train_durations_5 = []

for learning_rate in [0.1, 0.01, 0.001, 0.0001, 0.00001]:
    len_dataset = len(data.index.unique())
    num_target_variables = 1
    
    start_time = time.time()

    X_train, X_val, X_test, y_train, y_val, y_test = utils.train_test_val_data(df_scaled, len_dataset, num_target_variables)
    units_outputs = y_train.shape[1]
    
    model = Sequential()

    # Eingabeschicht
    model.add(GRU(units=units_0,            # Hyperparameter -> kann variiert und angepasst werden
                return_sequences=False,   # Konfigurationsparameter, default: False
                                            # Funktionalität:   bestimmt, ob die Schicht einen Ausgabevektor für jeden Zeitpunkt in der Eingabesequenz (return_sequences=True) 
                                            #                   oder nur für den letzten Zeitpunkt (return_sequences=False) zurückgeben soll
                                            #   False: gibt Ausgabevektor für den letzten Zeitschritt zurück: (Anzahl der Beispiele, Anzahl der Units)
                                            #   True:  gibt Ausgabevektor für jeden Zeitschritt in der Eingabesequenz zurück: (Anzahl der Beispiele, Anzahl der Zeitschritte, Anzahl der Units)
                                            # Anwendung: return_sequences=True: mehrere rekurrente Schichten hintereinander (damit jede Schicht eine Sequenz an die nächste weitergibt),  Ausgabe des Modells ist selbst eine Sequenz; 
                                            #            ansonsten: return_sequences=False.
                                            # 
                input_shape=(
                    X_train.shape[1],           # Sequenzlänge
                    X_train.shape[2]            # Anzahl der Features
                    )
                )
            )

    # Ausgabeschicht
    model.add(Dense(units_outputs, activation='linear'))

    optimizer = Adagrad(learning_rate=learning_rate)
    
    # Konfigurieren des Modells für das Training -> Festlegung der Lernart sowie die Bewertung des Trainingsprozesses
    model.compile(
        optimizer=optimizer,                # der Optimizer ist ein Algorithmus zur Aktualisierung des Netzwerks, wobei die Gewichte des Modells so angepasst werden, dass Verluste minimiert werden
                                            # Verschiedene Optimierer haben unterschiedliche Eigenschaften:
                                            # Adam - adaptive moment estimation: Grundprinzipien
                                            #   - Adaptive Lernraten: Lernrate wird für jeden Parameter individuell angepasst, basierend auf der Schätzung des ersten Mittelwert und des zweiten Moments der Gradienten
                                            #   - Moment-Schätzungen:   > erstes Moment (Mittelwert): Berechnung expontentiell abnehmender Durchschnittswerte vergangener Gradienten -> Steuerung zu relevanten Richtung des Gradientenabstiegs
                                            #                           > zweites Moment(Varianz): Berechnung exponentiuell abnehmender Durchschnittswerte vergangener quadrierter Gradienten 
                                            #                                                       -> Adaption der Lernrate, Regulierung der Schrittgröße basierend auf der Unsicherheit des Gradienten
                                            #   - Korrektur der Bias: Verhinderung der Tendenz, das Schätzungen zu Beginn gegen 0 gehen
                                            # Vorteile: Effizienz, wenigeer manuelle Einstellung der Lernrate, gute Performance bei großen Datenmengen/vielen Parametern
        loss='mean_squared_error',          # Verlustfunktion, misst die Genauigkeit des Modells. MSE misst die durchschnittliche quadratische Abweichung zwischen den vorhergesagten und den tatsächlichen Werten
        metrics=['mean_absolute_error']      # Metriken, die für das Training bewertet werden sollen, weitere Alternativen: 'accuracy', ...
        )
    
    # Trainieren des Modells
    history = model.fit(
        X_train, y_train,                   # Übergabe der Trainingsdaten
        epochs=50,                          # Anzahl der Durchläufe des gesamten Trainingsdatensatzes
                                            #   -> Einfluss: Mehr Epochen können zu einer besseren Anpassung des Modells führen <-> Gefahr des Overfittings
        batch_size=batch_size,                      # Bestimmt die Anzahl der verwendeten Datenpunkte für eine Iteration, bevor die Modellgewichte aktualisiert werden
                                            #   -> größere Batch-Größen: stabilere, aber langsamer konvergierende Updates <-> kleinere Batch-Größen: schnellere, weniger stabile Updates
        validation_data=(X_val, y_val),     # Validierungsdaten, ermöglichen die Überwachung des Trainingsprozesses -> Erkennung von Overfitting
        callbacks=[PlotLossesKeras()],
        use_multiprocessing=True,           # Laufzeitoptimierung
        workers=4,                          # Nutzen mehrerer CPU-Kerne
        verbose=1,                          # Steuert die Menge an Infos, welche während des Trainings ausgegeben werden -> verbose=1 zeigt den Fortschritt für jede Epoche an
    )

    learning_rate_2_curves.append(history)

    end_time = time.time()
    training_duration = end_time - start_time  # Dauer in Sekunden

    train_durations_5.append(training_duration)

with open('data/hyper_analysis/232_curves.pkl', 'wb') as f:
    pickle.dump(learning_rate_2_curves, f)

with open('data/hyper_analysis/232_durations.pkl', 'wb') as f:
    pickle.dump(train_durations_5, f)

In [ ]:
with open('data/hyper_analysis/232_curves.pkl', 'rb') as f:
    learning_rate_2_curves= pickle.load(f)

with open('data/hyper_analysis/232_durations.pkl', 'rb') as f:
    train_durations_5 = pickle.load(f)

plot_validation_losses_and_durations(learning_rate_2_curves, ["0.1", "0.01", "0.001", "0.0001", "0.00001"], train_durations_5)

#### 2.4 Einfluss Optimierer

In [ ]:
optimizer_curves = []
train_durations_4 = []

for optimizer in [Adadelta(learning_rate=0.01), SGD(learning_rate=0.01), Adagrad(learning_rate=0.01), Adam(learning_rate=0.0047)]:
    len_dataset = len(data.index.unique())
    num_target_variables = 1
    
    start_time = time.time()

    X_train, X_val, X_test, y_train, y_val, y_test = utils.train_test_val_data(df_scaled, len_dataset, num_target_variables)
    units_outputs = y_train.shape[1]
    
    model = Sequential()

    # Eingabeschicht
    model.add(GRU(units=units_0,            # Hyperparameter -> kann variiert und angepasst werden
                return_sequences=False,   # Konfigurationsparameter, default: False
                                            # Funktionalität:   bestimmt, ob die Schicht einen Ausgabevektor für jeden Zeitpunkt in der Eingabesequenz (return_sequences=True) 
                                            #                   oder nur für den letzten Zeitpunkt (return_sequences=False) zurückgeben soll
                                            #   False: gibt Ausgabevektor für den letzten Zeitschritt zurück: (Anzahl der Beispiele, Anzahl der Units)
                                            #   True:  gibt Ausgabevektor für jeden Zeitschritt in der Eingabesequenz zurück: (Anzahl der Beispiele, Anzahl der Zeitschritte, Anzahl der Units)
                                            # Anwendung: return_sequences=True: mehrere rekurrente Schichten hintereinander (damit jede Schicht eine Sequenz an die nächste weitergibt),  Ausgabe des Modells ist selbst eine Sequenz; 
                                            #            ansonsten: return_sequences=False.
                                            # 
                input_shape=(
                    X_train.shape[1],           # Sequenzlänge
                    X_train.shape[2]            # Anzahl der Features
                    )
                )
            )

    # Ausgabeschicht
    model.add(Dense(units_outputs, activation='linear'))
    
    # Konfigurieren des Modells für das Training -> Festlegung der Lernart sowie die Bewertung des Trainingsprozesses
    model.compile(
        optimizer=optimizer,                   # der Optimizer ist ein Algorithmus zur Aktualisierung des Netzwerks, wobei die Gewichte des Modells so angepasst werden, dass Verluste minimiert werden
                                            # Verschiedene Optimierer haben unterschiedliche Eigenschaften:
                                            # Adam - adaptive moment estimation: Grundprinzipien
                                            #   - Adaptive Lernraten: Lernrate wird für jeden Parameter individuell angepasst, basierend auf der Schätzung des ersten Mittelwert und des zweiten Moments der Gradienten
                                            #   - Moment-Schätzungen:   > erstes Moment (Mittelwert): Berechnung expontentiell abnehmender Durchschnittswerte vergangener Gradienten -> Steuerung zu relevanten Richtung des Gradientenabstiegs
                                            #                           > zweites Moment(Varianz): Berechnung exponentiuell abnehmender Durchschnittswerte vergangener quadrierter Gradienten 
                                            #                                                       -> Adaption der Lernrate, Regulierung der Schrittgröße basierend auf der Unsicherheit des Gradienten
                                            #   - Korrektur der Bias: Verhinderung der Tendenz, das Schätzungen zu Beginn gegen 0 gehen
                                            # Vorteile: Effizienz, wenigeer manuelle Einstellung der Lernrate, gute Performance bei großen Datenmengen/vielen Parametern
        loss='mean_squared_error',          # Verlustfunktion, misst die Genauigkeit des Modells. MSE misst die durchschnittliche quadratische Abweichung zwischen den vorhergesagten und den tatsächlichen Werten
        metrics=['mean_absolute_error']      # Metriken, die für das Training bewertet werden sollen, weitere Alternativen: 'accuracy', ...
        )
    
    # Trainieren des Modells
    history = model.fit(
        X_train, y_train,                   # Übergabe der Trainingsdaten
        epochs=50,                          # Anzahl der Durchläufe des gesamten Trainingsdatensatzes
                                            #   -> Einfluss: Mehr Epochen können zu einer besseren Anpassung des Modells führen <-> Gefahr des Overfittings
        batch_size=batch_size,                      # Bestimmt die Anzahl der verwendeten Datenpunkte für eine Iteration, bevor die Modellgewichte aktualisiert werden
                                            #   -> größere Batch-Größen: stabilere, aber langsamer konvergierende Updates <-> kleinere Batch-Größen: schnellere, weniger stabile Updates
        validation_data=(X_val, y_val),     # Validierungsdaten, ermöglichen die Überwachung des Trainingsprozesses -> Erkennung von Overfitting
        callbacks=[PlotLossesKeras()],
        use_multiprocessing=True,           # Laufzeitoptimierung
        workers=4,                          # Nutzen mehrerer CPU-Kerne
        verbose=1,                          # Steuert die Menge an Infos, welche während des Trainings ausgegeben werden -> verbose=1 zeigt den Fortschritt für jede Epoche an
    )

    optimizer_curves.append(history)

    end_time = time.time()
    training_duration = end_time - start_time  # Dauer in Sekunden

    train_durations_4.append(training_duration)

optimizer_curves_2 = optimizer_curves
optimizer_curves_2[3] = learning_rate_curves[1]
train_durations_4_2 = train_durations_4
train_durations_4_2[3] = train_durations_3[1]

with open('data/hyper_analysis/231_curves.pkl', 'wb') as f:
    pickle.dump(optimizer_curves_2, f)

with open('data/hyper_analysis/231_durations.pkl', 'wb') as f:
    pickle.dump(train_durations_4_2, f)

In [ ]:
with open('data/hyper_analysis/231_curves.pkl', 'rb') as f:
    optimizer_curves_2 = pickle.load(f)

with open('data/hyper_analysis/231_durations.pkl', 'rb') as f:
    train_durations_4_2 = pickle.load(f)

plot_validation_losses_and_durations(optimizer_curves_2, ["Adadelta", "SGD", "Adagrad", "Adam"], train_durations_4_2)

#### 2.5 Influence Regularization

In [ ]:
units_0 = 64
units_outputs = y_train.shape[1]
batch_size = 32

reg_curves = []
train_durations_5 = []

for l1l2 in [(0.001, 0.001), (0.0001,0.0001), (0.00001, 0.00001)]:
    len_dataset = len(data.index.unique())
    num_target_variables = 1
    
    start_time = time.time()

    X_train, X_val, X_test, y_train, y_val, y_test = utils.train_test_val_data(df_scaled, len_dataset, num_target_variables)
    units_outputs = y_train.shape[1]
    
    model = Sequential()

    # Eingabeschicht
    model.add(GRU(units=units_0,            # Hyperparameter -> kann variiert und angepasst werden
                return_sequences=False,   # Konfigurationsparameter, default: False
                                            # Funktionalität:   bestimmt, ob die Schicht einen Ausgabevektor für jeden Zeitpunkt in der Eingabesequenz (return_sequences=True) 
                                            #                   oder nur für den letzten Zeitpunkt (return_sequences=False) zurückgeben soll
                                            #   False: gibt Ausgabevektor für den letzten Zeitschritt zurück: (Anzahl der Beispiele, Anzahl der Units)
                                            #   True:  gibt Ausgabevektor für jeden Zeitschritt in der Eingabesequenz zurück: (Anzahl der Beispiele, Anzahl der Zeitschritte, Anzahl der Units)
                                            # Anwendung: return_sequences=True: mehrere rekurrente Schichten hintereinander (damit jede Schicht eine Sequenz an die nächste weitergibt),  Ausgabe des Modells ist selbst eine Sequenz; 
                                            #            ansonsten: return_sequences=False.
                                            # 
                input_shape=(
                    X_train.shape[1],           # Sequenzlänge
                    X_train.shape[2]            # Anzahl der Features
                    )
                )
            )

    # Ausgabeschicht
    model.add(Dense(units_outputs, 
                    activation='linear',
                    kernel_regularizer=l1_l2(l1=l1l2[0], l2=l1l2[1])
                    ))

    
    # Konfigurieren des Modells für das Training -> Festlegung der Lernart sowie die Bewertung des Trainingsprozesses
    model.compile(
        optimizer='adam',                # der Optimizer ist ein Algorithmus zur Aktualisierung des Netzwerks, wobei die Gewichte des Modells so angepasst werden, dass Verluste minimiert werden
                                            # Verschiedene Optimierer haben unterschiedliche Eigenschaften:
                                            # Adam - adaptive moment estimation: Grundprinzipien
                                            #   - Adaptive Lernraten: Lernrate wird für jeden Parameter individuell angepasst, basierend auf der Schätzung des ersten Mittelwert und des zweiten Moments der Gradienten
                                            #   - Moment-Schätzungen:   > erstes Moment (Mittelwert): Berechnung expontentiell abnehmender Durchschnittswerte vergangener Gradienten -> Steuerung zu relevanten Richtung des Gradientenabstiegs
                                            #                           > zweites Moment(Varianz): Berechnung exponentiuell abnehmender Durchschnittswerte vergangener quadrierter Gradienten 
                                            #                                                       -> Adaption der Lernrate, Regulierung der Schrittgröße basierend auf der Unsicherheit des Gradienten
                                            #   - Korrektur der Bias: Verhinderung der Tendenz, das Schätzungen zu Beginn gegen 0 gehen
                                            # Vorteile: Effizienz, wenigeer manuelle Einstellung der Lernrate, gute Performance bei großen Datenmengen/vielen Parametern
        loss='mean_squared_error',          # Verlustfunktion, misst die Genauigkeit des Modells. MSE misst die durchschnittliche quadratische Abweichung zwischen den vorhergesagten und den tatsächlichen Werten
        metrics=['mean_absolute_error']      # Metriken, die für das Training bewertet werden sollen, weitere Alternativen: 'accuracy', ...
        )
    
    # Trainieren des Modells
    history = model.fit(
        X_train, y_train,                   # Übergabe der Trainingsdaten
        epochs=50,                          # Anzahl der Durchläufe des gesamten Trainingsdatensatzes
                                            #   -> Einfluss: Mehr Epochen können zu einer besseren Anpassung des Modells führen <-> Gefahr des Overfittings
        batch_size=batch_size,                      # Bestimmt die Anzahl der verwendeten Datenpunkte für eine Iteration, bevor die Modellgewichte aktualisiert werden
                                            #   -> größere Batch-Größen: stabilere, aber langsamer konvergierende Updates <-> kleinere Batch-Größen: schnellere, weniger stabile Updates
        validation_data=(X_val, y_val),     # Validierungsdaten, ermöglichen die Überwachung des Trainingsprozesses -> Erkennung von Overfitting
        callbacks=[PlotLossesKeras()],
        use_multiprocessing=True,           # Laufzeitoptimierung
        workers=4,                          # Nutzen mehrerer CPU-Kerne
        verbose=1,                          # Steuert die Menge an Infos, welche während des Trainings ausgegeben werden -> verbose=1 zeigt den Fortschritt für jede Epoche an
    )

    reg_curves.append(history)

    end_time = time.time()
    training_duration = end_time - start_time  # Dauer in Sekunden

    train_durations_5.append(training_duration)

with open('data/hyper_analysis/25_curves.pkl', 'wb') as f:
    pickle.dump(reg_curves, f)

with open('data/hyper_analysis/25_durations.pkl', 'wb') as f:
    pickle.dump(train_durations_5, f)

In [ ]:
plot_validation_losses_and_durations(reg_curves, ["(10e-3, 10e-3)", "(10e-4, 10e-4)", "(10e-5, 10e-5)"], train_durations_5)

____

### 2. Hyperparameteroptimierung using Random Search

#### 2.1 P-Model

In [ ]:
class GRUHyperModel(HyperModel):
    '''
    Hypermodel used for Hyperparameteroptimization
    '''
    def __init__(self, input_shape, output_units):
        self.input_shape = input_shape
        self.output_units = output_units

    def build(self, hp):
        model = Sequential()
        
        # Input layer
        model.add(GRU(
            units=hp.get('units_0'),
            return_sequences=hp.get('num_gru_layers') > 1,
            input_shape=self.input_shape
        ))
        model.add(Dropout(hp.get('dropout_rate_0')))
        
        # Hidden Layer
        for i in range(1, hp.get('num_gru_layers')):
            model.add(GRU(
                units=hp.get(f'units_{i}'),
                return_sequences=i < hp.get('num_gru_layers') - 1
            ))
            model.add(Dropout(hp.get(f'dropout_rate_{i}')))

        # Output Layer
        model.add(Dense(self.output_units, 
                        activation='linear',
                        kernel_regularizer=l1_l2(l1=hp.get('l1'),
                                           l2=hp.get('l2'))), 
                  )
        model.compile(
        optimizer=Adam(hp.get('learning_rate')),
        loss='mean_squared_error'
    )
        return model
    
# definition of Hyperparameters
hp = HyperParameters()
hp.Int('num_gru_layers', 1, 5, default=2)
hp.Int('units_0', 32, 256, step=32)
hp.Float('dropout_rate_0', 0, 0.5, step=0.1)
for i in range(1, 5):  
    hp.Int(f'units_{i}', 32, 256, step=32, default=32)
    hp.Float(f'dropout_rate_{i}', 0, 0.5, step=0.1, default=0.1)
hp.Choice('batch_size', values=[32, 64, 128])
hp.Float('learning_rate', min_value=1e-4, 
        max_value=1e-2, sampling='LOG')
hp.Float('l1', min_value=1e-5, 
        max_value=1e-4, sampling='LOG')
hp.Float('l2', min_value=1e-5, 
        max_value=1e-4, sampling='LOG')

# input and output shape
input_shape = (None, 13) 
output_units = 96 

# initialize tuner
tuner = RandomSearch(
    GRUHyperModel(input_shape, output_units),
    hyperparameters=hp,
    objective='val_loss',
    max_trials=20,
    executions_per_trial=1,
    directory='data/tuner',
    project_name='gru_hyperparam_P'
)

# start sweep
tuner.search(
    x=X_train,
    y=y_train,
    epochs=10,
    validation_data=(X_val, y_val),
    batch_size=hp.get('batch_size')  
)

In [ ]:
from utils.utils import get_optimization_results

results = get_optimization_results(tuner, 100)
results = results[results['val_loss'] == results['val_loss'].min()]
results

#### 2.2 PF-Model

Load PF data

In [ ]:
data = data_loader(config.columns_PF)

# scale data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)
df_scaled = pd.DataFrame(scaled_data, columns=data.columns)

X_train, X_val, X_test, y_train, y_val, y_test = train_test_val_data(df_scaled, 
                                                                     len(data.index.unique()), 
                                                                     1)

In [ ]:
class GRUHyperModel(HyperModel):
    '''
    Hypermodel used for Hyperparameteroptimization
    '''
    def __init__(self, input_shape, output_units):
        self.input_shape = input_shape
        self.output_units = output_units

    def build(self, hp):
        model = Sequential()
        
        # Input layer
        model.add(GRU(
            units=hp.get('units_0'),
            return_sequences=hp.get('num_gru_layers') > 1,
            input_shape=self.input_shape
        ))
        model.add(Dropout(hp.get('dropout_rate_0')))
        
        # Hidden Layer
        for i in range(1, hp.get('num_gru_layers')):
            model.add(GRU(
                units=hp.get(f'units_{i}'),
                return_sequences=i < hp.get('num_gru_layers') - 1
            ))
            model.add(Dropout(hp.get(f'dropout_rate_{i}')))

        # Output Layer
        model.add(Dense(self.output_units, activation='linear'))
        model.compile(
        optimizer=Adam(hp.get('learning_rate')),
        loss='mean_squared_error'
    )
        return model
    
# definition of Hyperparameters
hp = HyperParameters()
hp.Int('num_gru_layers', 1, 5, default=2)
hp.Int('units_0', 32, 256, step=32)
hp.Float('dropout_rate_0', 0, 0.5, step=0.1)
for i in range(1, 5):  
    hp.Int(f'units_{i}', 32, 256, step=32, default=32)
    hp.Float(f'dropout_rate_{i}', 0, 0.5, step=0.1, default=0.1)
hp.Choice('batch_size', values=[32, 64, 128])
hp.Float('learning_rate', min_value=1e-4, 
        max_value=1e-2, sampling='LOG')

# input and output shape
input_shape = (None, 13) 
output_units = 96 

# initialize tuner
tuner = RandomSearch(
    GRUHyperModel(input_shape, output_units),
    hyperparameters=hp,
    objective='val_loss',
    max_trials=20,
    executions_per_trial=1,
    directory='data/tuner',
    project_name='gru_hyperparam_cosphi'
)

# start sweep
tuner.search(
    x=X_train,
    y=y_train,
    epochs=10,
    validation_data=(X_val, y_val),
    batch_size=hp.get('batch_size')  
)

In [ ]:
results = get_optimization_results(tuner, 100)
results = results[results['val_loss'] == results['val_loss'].min()]
results